# Tensorflow
This notebook is created to practice tensorflow library basics. Since I do know pytorch in a very good level, to create this notes, I used the beginning chapters of expert tutorials. You can find the tutorials from the website of [tensorflow](https://www.tensorflow.org/tutorials). After the beginning chapters of the tutorials were finished, I used tensorflow [guide](https://www.tensorflow.org/guide).

## Tensorflow Basics
TensorFlow is an end-to-end platform for machine learning. It supports the following:

- Multidimensional-array based numeric computation (similar to NumPy)
- GPU and distributed processing
- Automatic differentiation
- Model construction, training, and export
and more...

Tensorflow operates on multidimensional arrays or _tensors_ represented as **tf.Tensor** objects. The most important attributes of a __tf.Tensor__ is its _shape_ and _dtype_:
- __Tensor.shape:__ tells you the size of the tensor along each of its axes
- __Tensor.dtype:__ tels you the type of the all elements in the tensor

You can use standart mathematical operations on tensors, as well as many operations specialized for machine learning. Running to many parallel computation on CPU can be really slow. Hence, tensorflow is designed for supporting GPU computations.


In [6]:
# tensorflow tensor basics

import warnings
import tensorflow as tf

warnings.filterwarnings("ignore")

x = tf.constant([[1, 2, 3], [4, 5, 6]])
print(f"Tensor x -> {x}")
print(f"Shape of x -> {x.shape}")
print(f"Dtype of x -> {x.dtype}")

print("\nBasic Operations")
print("Square:")
print(x * x)
print("\n Multiplication (Matrix):")
print(x @ tf.transpose(x))
x = tf.cast(x, tf.float32)
print("\n Softmax operation:")
print(tf.nn.softmax(x, axis=-1))

Tensor x -> [[1 2 3]
 [4 5 6]]
Shape of x -> (2, 3)
Dtype of x -> <dtype: 'int32'>

Basic Operations
Square:
tf.Tensor(
[[ 1  4  9]
 [16 25 36]], shape=(2, 3), dtype=int32)

 Multiplication (Matrix):
tf.Tensor(
[[14 32]
 [32 77]], shape=(2, 2), dtype=int32)

 Softmax operation:
tf.Tensor(
[[0.09003057 0.24472848 0.66524094]
 [0.09003057 0.24472848 0.66524094]], shape=(2, 3), dtype=float32)


### Tensorflow Variables
Normal __tf.Tensor__ objects are immutable. To store weights (or mutable state) in TensorFlow, use __tf.Variable__. Everything is similar for __tf.Variable__ and __tf.Tensor__ objects except mutability.

### Autmatic differentiation
Gradient descent and related algorithms are a cornerstone of modern machine learning. To enable this, TensorFlow implements automatic differention (autodiff), which uses calculus to compute gradients. Typically, you'll use this to calculate the gradient of the model's error or loss with respect to its weights.

In [10]:
# using tensorflow variables
var = tf.Variable([0.0, 0.0, 0.0])
print("Variable:")
print(var)
var.assign([1, 2, 3])
print("\nVariable after assignment:")
print(var)
var.assign_add([3, 2, 1])
print("\nVariable after addition:")
print(var)

# using automatic differentiation
x = tf.Variable(1.0)

def f(x: tf.Variable):
    return x**2 + 2*x - 5

print("\nFunction without gradient:")
print(f(x))
print("\nFunction with gradient")
with tf.GradientTape() as tape:
    y = f(x)

g_x = tape.gradient(y ,x) # g(x) = dy/dx
print(f(x))
print(g_x)

Variable:
<tf.Variable 'Variable:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>

Variable after assignment:
<tf.Variable 'Variable:0' shape=(3,) dtype=float32, numpy=array([1., 2., 3.], dtype=float32)>

Variable after addition:
<tf.Variable 'Variable:0' shape=(3,) dtype=float32, numpy=array([4., 4., 4.], dtype=float32)>

Function without gradient:
tf.Tensor(-2.0, shape=(), dtype=float32)

Function with gradient
tf.Tensor(-2.0, shape=(), dtype=float32)
tf.Tensor(4.0, shape=(), dtype=float32)


### Graphs and Functions 
While you can use TensorFlow interactively like any Python library, TensorFlow also provides tools for:

- __Performance optimization:__ to speed up training and inference.
- __Export:__ so you can save your model when it's done training.

These requires that you use __tf.Function__ to separate your pure-Tensorflow code from Python.

The first time you run the __tf.function__, although it executes in Python, it captures a complete, optimized graph representing the TensorFlow computations done within the function.

On subsequent calls TensorFlow only executes the optimized graph, skipping any non-TensorFlow steps. Below, note that my_func doesn't print tracing since print is a Python function, not a TensorFlow function.

A graph may not be reusable for inputs with a different signature (shape and dtype), so a new graph is generated instead. These captured graphs provide two benefits:

- In many cases they provide a significant speedup in execution.
- You can export these graphs, using tf.saved_model, to run on other systems like a server or a mobile device, no Python installation required.

In [13]:
# tf.function implementation
@tf.function
def my_func(x):
  print('Tracing.')
  return tf.reduce_sum(x)

# first call to generate graph
x = tf.constant([1, 2, 3])
print(my_func(x), "\n")

# second call (ignoring the non-Tensorflow steps)
x = tf.constant([10, 9, 8])
print(my_func(x), "\n")

# generating a new graph for different dtype
x = tf.constant([10.1, 9.8, 8.1], dtype=tf.float32)
print(my_func(x))


Tracing.
tf.Tensor(6, shape=(), dtype=int32) 

tf.Tensor(27, shape=(), dtype=int32) 

Tracing.
tf.Tensor(28.000002, shape=(), dtype=float32)


### Modules, Layers and Models
__tf.Module__ is a class for managing your __tf.Variable__ objects, and the __tf.function__ objects that operate on them. The __tf.Module__ class is necessary to support two significant features:

1. You can save and restore the values of your variables using __tf.train.Checkpoint__. This is useful during training as it is quick to save and restore a model's state.
2. You can import and export the __tf.Variable__ values and the __tf.function__ graphs using __tf.saved_model__. This allows you to run your model independently of the Python program that created it.

Here is a complete example exporting a simple tf.Module object:

In [16]:
class MyModule(tf.Module):
  def __init__(self, value):
    self.weight = tf.Variable(value)

  @tf.function
  def multiply(self, x):
    return x * self.weight

mod = MyModule(3)
print("Module output:")
print(mod.multiply(tf.constant([1, 2, 3])))

# For saving the module
# save_path = './saved'
# tf.saved_model.save(mod, save_path)

# For loading the module
# reloaded = tf.saved_model.load(save_path)
# reloaded.multiply(tf.constant([1, 2, 3]))

Module output:
tf.Tensor([3 6 9], shape=(3,), dtype=int32)
